In [10]:
import pandas as pd
import numpy as np
import textacy
import json
from tqdm import tqdm 

In [4]:
a = pd.read_csv("data/transcripts.csv")

In [7]:
corpus=[]
docs = a['transcript'].tolist()[:200]
for doc in tqdm(docs):
    corpus.append(textacy.Doc(content=doc.decode('utf-8'),lang=u'en'))

100%|██████████| 200/200 [03:09<00:00,  1.05it/s]


## BOW Matrix extracted from Textacy (Pass to Baseline model)

In [8]:
tokenized_docs = (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True) for doc in corpus)
vectorizer = textacy.Vectorizer(apply_idf=True, norm='l2',min_df=3, max_df=0.95)
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)
doc_term_matrix

<200x6241 sparse matrix of type '<type 'numpy.float64'>'
	with 90633 stored elements in Compressed Sparse Row format>

## Get Labels

In [12]:
metadata = pd.read_csv("data/ted_main.csv")

In [13]:
arr = json.loads(metadata['ratings'][0].replace("'",'\"'))

In [14]:
def combine_list_of_dicts(arr):
    dict_ratings = {}
    for a in arr:
        if a["id"] not in dict_ratings:
            dict_ratings[a["id"]]=a
            
    return dict_ratings

d = combine_list_of_dicts(arr)

In [15]:
sorted(d.items(), key= lambda x:x[1]["count"],reverse=True)

[(10, {u'count': 24924, u'id': 10, u'name': u'Inspiring'}),
 (7, {u'count': 19645, u'id': 7, u'name': u'Funny'}),
 (24, {u'count': 10704, u'id': 24, u'name': u'Persuasive'}),
 (22, {u'count': 10581, u'id': 22, u'name': u'Fascinating'}),
 (8, {u'count': 7346, u'id': 8, u'name': u'Informative'}),
 (9, {u'count': 6073, u'id': 9, u'name': u'Ingenious'}),
 (1, {u'count': 4573, u'id': 1, u'name': u'Beautiful'}),
 (23, {u'count': 4439, u'id': 23, u'name': u'Jaw-dropping'}),
 (3, {u'count': 3253, u'id': 3, u'name': u'Courageous'}),
 (25, {u'count': 1174, u'id': 25, u'name': u'OK'}),
 (11, {u'count': 387, u'id': 11, u'name': u'Longwinded'}),
 (21, {u'count': 300, u'id': 21, u'name': u'Unconvincing'}),
 (2, {u'count': 242, u'id': 2, u'name': u'Confusing'}),
 (26, {u'count': 209, u'id': 26, u'name': u'Obnoxious'})]

In [16]:
def getTEDRating(string):
    arr = json.loads(string.replace("'",'\"'))
    d = combine_list_of_dicts(arr) 
#     print(d.items())
    return sorted(d.items(), key= lambda x:x[1]["count"],reverse=True)[0][1]['id']
getTEDRating(metadata['ratings'][0])

10

In [17]:
labels = [getTEDRating(x) for x in metadata['ratings'].tolist()[:200]]

In [18]:
seriess = {'transcripts': pd.Series(a['transcript']),
    'labels': pd.Series(labels)}
dataset = pd.DataFrame(seriess)

In [19]:
dataset

,labels,transcripts
0,10.0,Good morning. How are you?(Laughter)It's been ...
1,7.0,"Thank you so much, Chris. And it's truly a gre..."
2,7.0,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,10.0,If you're here today — and I'm very happy that...
4,8.0,"About 10 years ago, I took on the task to teac..."
5,10.0,Thank you. I have to tell you I'm both challen...
6,7.0,"On September 10, the morning of my seventh bir..."
7,9.0,I'm going to present three projects in rapid f...
8,24.0,It's wonderful to be back. I love this wonderf...
9,10.0,"I'm often asked, ""What surprised you about the..."
